In [ ]:
!pip install mutagen
!pip install transforms3d
import tensorflow as tf
from PIL import Image, ImageDraw
from google.colab import drive
import os
import matplotlib.pyplot as plt
import sys
import importlib
import shutil
import csv
import pandas as pd
import random
import cProfile
import pstats
import numpy as np
drive.mount('/content/drive')
# !rm -rf subject_txt
# !rm -rf all_asfamc
!unzip /content/drive/My\ Drive/Colab\ Notebooks/Pose\ Priors/data/allasfamc.zip
!unzip /content/drive/My\ Drive/Colab\ Notebooks/Pose\ Priors/data/subject_txt.zip
# import amc_parser as a
import contextlib

     |████████████████████████████████| 225kB 5.2MB/s 
     |████████████████████████████████| 71kB 3.1MB/s 
  Created wheel for transforms3d: filename=transforms3d-0.3.1-cp36-none-any.whl size=59374 sha256=7090756bb1cb44fb8ba4cdda9fce8715c74c121c49c5d3bb8df985f1dbd8e2e0
  Stored in directory: /root/.cache/pip/wheels/3d/3c/84/28d36677f3c760c048bd02b5a547ea0c4027770cc9cdb9af1e
Successfully built transforms3d
Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/drive
Archive:  /content/drive/My Drive/Colab Notebooks/Pos

In [ ]:
subjectlist = ['39',
 '140',
 '133',
 '134',
 '63',
 '06',
 '55',
 '60',
 '89',
 '38',
 '11',
 '25',
 '94',
 '108',
 '62',
 '76',
 '136',
 '49',
 '05',
 '45',
 '02',
 '23',
 '35',
 '86',
 '118',
 '64',
 '141',
 '74',
 '21',
 '01',
 '09',
 '08',
 '33',
 '07',
 '13',
 '14',
 '47',
 '37',
 '81',
 '18',
 '70',
 '28',
 '87',
 '77',
 '73',
 '41',
 '24',
 '132',
 '43',
 '142',
 '117',
 '120',
 '84',
 '31',
 '122',
 '12',
 '10',
 '88',
 '83',
 '30',
 '16',
 '82',
 '121',
 '104',
 '03',
 '114',
 '126',
 '32',
 '69',
 '113',
 '93',
 '135',
 '103',
 '91',
 '20',
 '128',
 '26',
 '125',
 '127',
 '106',
 '15',
 '29',
 '124',
 '17',
 '85',
 '36',
 '27',
 '42',
 '78',
 '40',
 '115',
 '137',
 '138',
 '102',
 '80',
 '46',
 '90',
 '22',
 '105',
 '143',
 '56',
 '34',
 '61',
 '79',
 '54',
 '131',
 '107',
 '123',
 '111',
 '139',
 '75',
 '19']

In [ ]:
def genData(subject, makeImgs, jointlist):
  sub = "{:0>2d}".format(subject)
  basepath = '/content/drive/My Drive/Colab Notebooks/Pose Priors/'
  subpath = '/content/all_asfamc/subjects/'+sub+'/'
  paths = checkPaths(sub, makeImgs, basepath)
  # try: 
  if makeImgs: csvPath, imgPath, gifPath = paths 
  else: csvPath = paths
  trials = sorted(os.listdir(subpath)) #ind 0 will be asf, rest amc
  for ind in range(len(trials)):
    trial = trials[ind]
    if ind == 0:
      asf_path = subpath+trial
      joints = a.parse_asf(asf_path)
      continue #don't do any amc processing
    trial_num = "{:0>2d}".format(ind) #for amc files
    amc_path = subpath+trial
    motions = a.parse_amc(amc_path) #get all motions in amc file
    # output = pd.DataFrame(columns=jointlist)
    output = np.zeros((len(motions), len(jointlist)*3))
    if makeImgs and ind == 1: #save first trial for indicated subjects
        im = Image.new('RGB', (200,200))
        imgpath = imgPath+trial_num+'/'
        os.mkdir(imgpath)
    for idx in range(len(motions)): #make all images in trial
        mot = "{:0>6d}".format(idx)
        joints['root'].set_motion(motions[idx])
        # print(joints['root'].coordinate)
        # lambda l: [item for sublist in l for item in sublist]
        flattened = [val[0]
                     for sublist in [joints[joint].coordinate 
                     for joint in jointlist]
                     for val in sublist] 
        if makeImgs and ind == 1:
          joints['root'].draw(imgpath+mot+'.png')
        # output = output.append(coordinates, ignore_index=True)
        # output[idx] = np.squeeze(np.asarray(coordinates), axis=2)
        output[idx] = np.asarray(flattened)
    
    # output.to_csv(csvPath+trial_num+'.csv')
    np.savetxt(csvPath+trial_num+".csv", output, delimiter=",")
    # print("saved")
    if makeImgs and ind == 1:
      gifpath = gifPath+trial_num+'.gif'
      makeGif(gifpath, imgpath)
  # except: 
  #   print(sub+" had an error.")
  #   print(sys.exc_info()[0])

In [ ]:
def checkPaths(sub, makeImgs, basepath):
  csvPath = basepath+'data/'+sub+'/'
  imgPath = basepath+'imgs/'+sub+'/'
  gifPath = basepath+'gifs/'+sub+'/'
  if os.path.exists(imgPath):
    shutil.rmtree(imgPath)
  if os.path.exists(gifPath):
    shutil.rmtree(gifPath)
  if os.path.exists(csvPath):
    shutil.rmtree(csvPath)
  os.mkdir(csvPath)
  if makeImgs: #check if gif/imgs directories exist - delete if they do
    os.mkdir(imgPath)
    os.mkdir(gifPath)
    return csvPath, imgPath, gifPath
  else: return csvPath

In [ ]:
makeGif('/content/drive/My Drive/Colab Notebooks/Pose Priors/gifs/17/01.gif', '/content/drive/My Drive/Colab Notebooks/Pose Priors/imgs/17/01/')

NameError: ignored

In [ ]:
def makeGif(gifpath, imgpath):
  figs = sorted(os.listdir(imgpath))
  frames = []
  for i in figs:
      new_frame = Image.open(imgpath+i)
      frames.append(new_frame)
  # Save into a GIF file that loops forever
  frames[0].save(gifpath, format='GIF',
          append_images=frames[1:],
            save_all=True, duration=len(frames)/120) #same as 120 fps but not sure how accurate fps setting is

In [ ]:
def makeSequence(subject, amc, desc, jointlist, xyz_header):

  sub = "{:0>2d}".format(subject)
  trial_num = amc[amc.find('_')+1:amc.find('.')]
  trial = int(trial_num)
  amcpath = '/content/all_asfamc/subjects/'+sub+'/'+amc
  asfpath = '/content/all_asfamc/subjects/'+sub+'/'+sub+'.asf'
  f = open(asfpath, "r")
  asf = f.read()
  test_motions = a.parse_amc(amcpath)
  angles = test_motions[0].keys()
  angle_header = []
  endings = ['-x', '-y', '-z', '-x2', '-y2', '-z2']
  for angle in angles:
    for i in range(len(test_motions[0][angle])):
      angle_header.append(angle+endings[i])
  angle_header_encoded = [x.encode('utf-8') for x in angle_header]
  xyz_header_encoded = [x.encode('utf-8') for x in xyz_header]

  id = str(hash(sub+'/'+trial_num))+'/'+sub+'/'+trial_num

  ex = tf.train.SequenceExample()
  ex.context.feature["example/id"].bytes_list.value.append(id.encode("utf8"))
  ex.context.feature["example/asf"].bytes_list.value.append(asf.encode("utf8"))
  ex.context.feature["example/angle_header"].bytes_list.value.extend(angle_header_encoded)
  ex.context.feature["example/xyz_header"].bytes_list.value.extend(xyz_header_encoded)
  ex.context.feature["example/description"].bytes_list.value.append(desc.encode("utf8"))

  timestamp = [int(i/120*1000000) for i in range(len(test_motions))]
  anglelist = [[motion[l] for l in angles] for motion in test_motions]
  angles = [[item for sublist in r for item in sublist] for r in anglelist]

  headers_nested = sequenceHelper(subject, trial, jointlist)
  for t in timestamp:
    ex.feature_lists.feature_list["ALL_JOINT_ANGLES/feature/timestamp"].feature.add().int64_list.value.append(t)
    ex.feature_lists.feature_list["ALL_JOINT_XYZ/feature/timestamp"].feature.add().int64_list.value.append(t)
  for angle in angles:
    ex.feature_lists.feature_list["ALL_JOINT_ANGLES/feature/floats"].feature.add().float_list.value[:] = angle
  for header in headers_nested:
    ex.feature_lists.feature_list["ALL_JOINT_XYZ/feature/floats"].feature.add().float_list.value[:] = header

  return ex 

In [ ]:
def sequenceHelper(subject, t, jointlist):
  sub = "{:0>2d}".format(subject)
  trial = "{:0>2d}".format(t) #for amc files
  subpath = '/content/all_asfamc/subjects/'+sub+'/'
  asf_path = subpath+sub+'.asf'
  amc_path = subpath+sub+'_'+trial+'.amc'
  joints = a.parse_asf(asf_path)
  motions = a.parse_amc(amc_path) #get all motions in amc file
  output = np.zeros((len(motions), len(jointlist)*3))
  for idx in range(len(motions)): #make all images in trial
    mot = "{:0>6d}".format(idx)
    joints['root'].set_motion(motions[idx])
    flattened = [val[0]
        for sublist in [joints[joint].coordinate 
        for joint in jointlist]
        for val in sublist] 
    output[idx] = np.asarray(flattened)
  return output

In [ ]:
def getDescription(sub, trial_amc):
  filename = "/content/subject_txt/subject" + str(sub) + ".txt" 
  d = {}
  with open(filename) as fin:
      next(fin) 
      next(fin)
      next(fin)
      for line in fin:
        try:
          ind = line.find('.avi')+5
          words = line.split()
          amc = [word for word in words if "amc" in word][0]
          d[amc] = line[ind:]
        except: pass
      return d[trial_amc]

In [ ]:
def _generate_single_example(subject, trial_amc):
    """For each row in the annotation CSV, generates the corresponding metadata.
    Args:
      subject: subject number (integer)
      trial_amc: amc filename 
    Yields:
      Each tf.SequenceExample of metadata
    """

    xyz_header = []
    jointlist = ['root', 'lhipjoint', 'lfemur', 'ltibia', 'lfoot', 'ltoes', 'rhipjoint', 'rfemur', 'rtibia', 'rfoot', 'rtoes', 'lowerback', 'upperback', 'thorax', 'lowerneck', 'upperneck', 'head', 'lclavicle', 'lhumerus', 'lradius', 'lwrist', 'lhand', 'lfingers', 'lthumb', 'rclavicle', 'rhumerus', 'rradius', 'rwrist', 'rhand', 'rfingers', 'rthumb']
    for joint in jointlist:
      xyz_header.append(joint+'-x')
      xyz_header.append(joint+'-y')
      xyz_header.append(joint+'-z')
    description = getDescription(subject, trial_amc)
    metadata = makeSequence(subject, trial_amc, description, jointlist, xyz_header)     
    return metadata

In [ ]:
def generate_splits():
    SPLITS = {}
    add = ["train", "test"]
    SPLITS["train"] = []
    SPLITS["test"] = []
    path = '/content/all_asfamc/subjects/'
    for subjectfolder in os.listdir(path):
      amcs = sorted(os.listdir(path+subjectfolder))[1:]
      for ind in range(len(amcs)):
        SPLITS[add[ind % 2]].append(amcs[ind]) 
    return SPLITS

In [ ]:
class MotionDataset(object):
  """Generates and loads a motion capture data set."""

  def __init__(self, path_to_data):
    if not path_to_data:
      raise ValueError("You must supply the path to the data directory.")
    self.path_to_data = path_to_data


  def as_dataset(self,
                 split,
                 shuffle=False,
                 repeat=False,
                 serialized_prefetch_size=32,
                 decoded_prefetch_size=32):
    """Returns the dataset as a tf.data.Dataset.
    Args:
      split: either "train" or "test"
      shuffle: if true, shuffles both files and examples.
      repeat: if true, repeats the data set forever.
      serialized_prefetch_size: the buffer size for reading from disk.
      decoded_prefetch_size: the buffer size after decoding.
    Returns:
      A tf.data.Dataset object with the following structure: {
        "images": uint8 tensor, shape [time, height, width, channels]
        "labels": one hot encoded label tensor, shape [2]
        "id": a unique string id for each example, shape []
      }
    """

    def parse_fn(sequence_example):
      #replace keys with strings
      context_features = {
          "example/id": tf.io.FixedLenFeature(shape=(), dtype=tf.string) ,
          "example/asf": 
              tf.io.FixedLenFeature(shape=(), dtype=tf.string),
          "example/angle_header": 
              tf.io.FixedLenFeature(shape=(62,), dtype=tf.string),
          "example/xyz_header": 
              tf.io.FixedLenFeature(shape=(93,), dtype=tf.string),
          "example/description": 
              tf.io.FixedLenFeature(shape=(), dtype=tf.string),
      }

      sequence_features = {
          "ALL_JOINT_ANGLES/feature/timestamp": 
              tf.io.FixedLenSequenceFeature(shape=(), dtype=tf.int64),
          "ALL_JOINT_XYZ/feature/timestamp": 
              tf.io.FixedLenSequenceFeature(shape=(), dtype=tf.int64),  
          "ALL_JOINT_ANGLES/feature/floats": 
              tf.io.FixedLenSequenceFeature(shape=(62,), dtype=tf.float32),
          "ALL_JOINT_XYZ/feature/floats": 
              tf.io.FixedLenSequenceFeature(shape=(93,), dtype=tf.float32),
      }
      
      parsed_context, parsed_sequence = tf.io.parse_single_sequence_example(
          sequence_example, context_features, sequence_features)
      return parsed_sequence

    SPLITS = generate_splits()
    print(SPLITS)
    if split not in SPLITS:
      raise ValueError("split '%s' is unknown." % split)
    all_shards = tf.io.gfile.glob(
        os.path.join(self.path_to_data, TF_RECORD_PATTERN % split + "-*-of-*"))
    if shuffle:
      random.shuffle(all_shards)
    all_shards_dataset = tf.data.Dataset.from_tensor_slices(all_shards)
    cycle_length = min(32, len(all_shards))
    dataset = all_shards_dataset.interleave( ##
        tf.data.TFRecordDataset,
            cycle_length=cycle_length,
            block_length=1)
    dataset = dataset.prefetch(serialized_prefetch_size)
    if shuffle:
      dataset = dataset.shuffle(serialized_prefetch_size)
    if repeat:
      dataset = dataset.repeat()
    dataset = dataset.map(parse_fn)
    dataset = dataset.prefetch(decoded_prefetch_size)
    return dataset

  def generate_examples(self):
    """Downloads data and generates sharded TFRecords.
    Downloads the data files, generates metadata, and processes the metadata
    with MediaPipe to produce tf.SequenceExamples for training. The resulting
    files can be read with as_dataset(). After running this function the
    original data files can be deleted.
    Args:
      path_to_mediapipe_binary: Path to the compiled binary for the BUILD target
        mediapipe/examples/desktop/demo:media_sequence_demo.
      path_to_graph_directory: Path to the directory with MediaPipe graphs in
        mediapipe/graphs/media_sequence/.
    """
    SPLITS = generate_splits()
    print(SPLITS["train"])
    # for split in SPLITS:
    split = "train"
    for j in range(1):
      all_metadata = []
      for trial_amc in SPLITS[split]:
        #get subject and trial_amc, append to all_metadata
        sub = int(trial_amc[:trial_amc.find('_')])
        metadata = (sub, trial_amc)
        all_metadata.append(metadata)
      random.seed(47)
      random.shuffle(all_metadata) #get pairs in split
      shard_names = [self._indexed_shard(split, i) for i in range(NUM_SHARDS)]
      print(shard_names)
      writers = [tf.io.TFRecordWriter(shard_name) for shard_name in shard_names]
      with _close_on_exit(writers) as writers:
        for i, (sub, trial_amc) in enumerate(all_metadata):
          try:
            seq_ex = _generate_single_example(sub, trial_amc)
            writers[i % len(writers)].write(seq_ex.SerializeToString())
          # print(writers[i % len(writers)])
          except: 
            print("Something went wrong.")
            print(sub, trial_amc)

  def _indexed_shard(self, split, index):
    """Constructs a sharded filename."""
    return os.path.join(
        self.path_to_data,
        TF_RECORD_PATTERN % split + "-%05d-of-%05d" % (index, NUM_SHARDS))
    
@contextlib.contextmanager
def _close_on_exit(writers):
  """Call close on all writers on exit."""
  try:
    yield writers
  finally:
    for writer in writers:
      writer.close()

In [ ]:
NUM_SHARDS = 10 # ~200MB/file
TF_RECORD_PATTERN = "mocap_dataset_%s_tfrecord"

In [ ]:
mocap = MotionDataset("/content/drive/My Drive/Colab Notebooks/Pose Priors")
mocap.generate_examples()

['39_01.amc', '39_03.amc', '39_05.amc', '39_07.amc', '39_09.amc', '39_11.amc', '39_13.amc', '140_01.amc', '140_03.amc', '140_05.amc', '140_07.amc', '140_09.amc', '133_01.amc', '133_03.amc', '133_05.amc', '133_07.amc', '133_09.amc', '133_11.amc', '133_13.amc', '133_15.amc', '133_17.amc', '133_19.amc', '133_21.amc', '133_23.amc', '133_25.amc', '134_01.amc', '134_03.amc', '134_05.amc', '134_07.amc', '134_09.amc', '134_11.amc', '134_13.amc', '134_15.amc', '63_01.amc', '63_03.amc', '63_05.amc', '63_07.amc', '63_09.amc', '63_11.amc', '63_13.amc', '63_15.amc', '63_17.amc', '63_19.amc', '63_21.amc', '63_23.amc', '63_25.amc', '63_27.amc', '63_29.amc', '06_01.amc', '06_03.amc', '06_05.amc', '06_07.amc', '06_09.amc', '06_11.amc', '06_13.amc', '06_15.amc', '55_01.amc', '55_03.amc', '55_05.amc', '55_07.amc', '55_09.amc', '55_11.amc', '55_13.amc', '55_15.amc', '55_17.amc', '55_19.amc', '55_21.amc', '55_23.amc', '55_25.amc', '55_27.amc', '60_01.amc', '60_03.amc', '60_05.amc', '60_07.amc', '60_09.amc'

In [ ]:
import amc_parser as a

In [ ]:
train = mocap.as_dataset("train")
test = mocap.as_dataset("test")
# xyz = next(iter(dataset))
# xyz['ALL_JOINT_XYZ/feature/floats'] #processing operations -> parse_fn 

{'train': ['39_01.amc', '39_03.amc', '39_05.amc', '39_07.amc', '39_09.amc', '39_11.amc', '39_13.amc', '140_01.amc', '140_03.amc', '140_05.amc', '140_07.amc', '140_09.amc', '133_01.amc', '133_03.amc', '133_05.amc', '133_07.amc', '133_09.amc', '133_11.amc', '133_13.amc', '133_15.amc', '133_17.amc', '133_19.amc', '133_21.amc', '133_23.amc', '133_25.amc', '134_01.amc', '134_03.amc', '134_05.amc', '134_07.amc', '134_09.amc', '134_11.amc', '134_13.amc', '134_15.amc', '63_01.amc', '63_03.amc', '63_05.amc', '63_07.amc', '63_09.amc', '63_11.amc', '63_13.amc', '63_15.amc', '63_17.amc', '63_19.amc', '63_21.amc', '63_23.amc', '63_25.amc', '63_27.amc', '63_29.amc', '06_01.amc', '06_03.amc', '06_05.amc', '06_07.amc', '06_09.amc', '06_11.amc', '06_13.amc', '06_15.amc', '55_01.amc', '55_03.amc', '55_05.amc', '55_07.amc', '55_09.amc', '55_11.amc', '55_13.amc', '55_15.amc', '55_17.amc', '55_19.amc', '55_21.amc', '55_23.amc', '55_25.amc', '55_27.amc', '60_01.amc', '60_03.amc', '60_05.amc', '60_07.amc', '

In [ ]:
#@title Import { display-mode: "form" }


import numpy as np

import tensorflow.compat.v2 as tf
tf.enable_v2_behavior()

import tensorflow_datasets as tfds
import tensorflow_probability as tfp


tfk = tf.keras
tfkl = tf.keras.layers
tfpl = tfp.layers
tfd = tfp.distributions

In [ ]:
# datasets, datasets_info = tfds.load(name='mnist',
#                                     with_info=True,
#                                     as_supervised=False)

def _preprocess(sample):
  # xyz = tf.cast(sample['ALL_JOINT_XYZ/feature/floats'], tf.float32) / 255.  # Scale to unit interval.
  xyz = sample['ALL_JOINT_XYZ/feature/floats']
  xyz = xyz[tf.random.uniform((), minval=0, maxval=tf.shape(xyz)[0], dtype=tf.int32)]
  # xyz = tf.cast(xyz < tf.random.uniform(tf.shape(xyz)), tf.float32)   # Randomly binarize.
  # assert not np.any(np.isnan(xyz.as_numpy())
  return xyz #,xyz

train_dataset = (train #train
                 .map(_preprocess)
                 .batch(64)
                 .prefetch(tf.data.experimental.AUTOTUNE)
                 .shuffle(int(10e3)))
test_dataset = (test
                .map(_preprocess)
                .batch(64)
                .prefetch(tf.data.experimental.AUTOTUNE))

In [ ]:
xyz = next(iter(test))
input_shape = xyz['ALL_JOINT_XYZ/feature/floats'].shape[1]
encoded_size = 22
base_depth = 31

In [ ]:
prior = tfd.Independent(tfd.Normal(loc=tf.zeros(encoded_size), scale=1),
                        reinterpreted_batch_ndims=1)

In [ ]:
encoder = tfk.Sequential([
    tfkl.InputLayer(input_shape=input_shape),
    tfkl.Dense(base_depth, activation=tf.nn.leaky_relu),
    tfkl.Dense(2 * base_depth, activation=tf.nn.leaky_relu),
    tfkl.Dense(4 * encoded_size, activation=tf.nn.leaky_relu),
    tfkl.Dense(tfpl.MultivariateNormalTriL.params_size(encoded_size),
               activation=None),
    tfpl.MultivariateNormalTriL(
        encoded_size,
        activity_regularizer=tfpl.KLDivergenceRegularizer(prior)),
], name="encoder")

In [ ]:
decoder = tfk.Sequential([ #activation function, multipliers
    tfkl.InputLayer(input_shape=[encoded_size]),
    tfkl.Dense(2 * base_depth, activation=tf.nn.leaky_relu),
    tfkl.Dense(input_shape, activation=tf.nn.leaky_relu),
    tfkl.Dense(tfpl.IndependentNormal.params_size(input_shape), activation=None),
    tfpl.IndependentNormal(input_shape, tfd.Distribution.mean),
], name="decoder")

In [ ]:
vae = tfk.Model(inputs=encoder.inputs,
                outputs=decoder(encoder.outputs[0]))

In [ ]:
negloglik = lambda x, rv_x: -rv_x.log_prob(x)

vae.compile(optimizer=tf.optimizers.Adam(learning_rate=3e-3), #change lr
            loss=negloglik)

_ = vae.fit(train_dataset, #change later - overfit
            epochs=50,
            validation_data=test_dataset)

Epoch 1/50
20/20 [==============================] - 5s 266ms/step - loss: 424.8678 - val_loss: 44.2966
Epoch 2/50
20/20 [==============================] - 5s 254ms/step - loss: 24.8180 - val_loss: 13.2159
Epoch 3/50
20/20 [==============================] - 5s 257ms/step - loss: 8.1525 - val_loss: 5.9066
Epoch 4/50
20/20 [==============================] - 5s 232ms/step - loss: 4.8405 - val_loss: 4.0863
Epoch 5/50
20/20 [==============================] - 5s 233ms/step - loss: 3.2786 - val_loss: 3.1104
Epoch 6/50
20/20 [==============================] - 5s 229ms/step - loss: 2.5774 - val_loss: 2.6708
Epoch 7/50
20/20 [==============================] - 5s 232ms/step - loss: 2.1938 - val_loss: 2.0945
Epoch 8/50
20/20 [==============================] - 5s 231ms/step - loss: 2.0994 - val_loss: 1.6978
Epoch 9/50
20/20 [==============================] - 5s 233ms/step - loss: 1.9548 - val_loss: 1.9905
Epoch 10/50
20/20 [==============================] - 5s 229ms/step - loss: 1.4913 - val_loss: 1

In [97]:
# We'll just examine ten random digits.
x = next(iter(test_dataset))
xhat = vae(x[0:1])
assert isinstance(xhat, tfd.Distribution)

In [99]:
xhat.mean()
#take samples
#plot

<tf.Tensor: shape=(1, 93), dtype=float32, numpy=
array([[ 0.00807456,  0.33750704, -0.1963158 , -0.3870178 , -0.21913552,
        -0.05958324, -0.20965359,  0.02101879, -0.19021869,  0.30128178,
         0.1296043 ,  0.17107885, -0.09635354,  0.31031495,  0.11145411,
        -0.0515461 , -0.18322645,  0.08011777, -0.20328808, -0.2296057 ,
        -0.07798928, -0.30993426,  0.00652018,  0.21515176, -0.17464814,
         0.212465  ,  0.20910384,  0.0738266 ,  0.0424258 , -0.09359562,
         0.29808947,  0.1429469 , -0.12669177,  0.1114177 ,  0.28257638,
        -0.22470629,  0.21609676, -0.13870656,  0.21588647, -0.04793104,
        -0.34154734,  0.33956107, -0.16463499, -0.03051628, -0.19989681,
        -0.10695566,  0.2557865 , -0.09707221, -0.20866233,  0.34856972,
         0.1554958 ,  0.05592293, -0.19985798,  0.01943134,  0.15518595,
         0.02997934, -0.18906589,  0.21084107,  0.18752825,  0.1922783 ,
        -0.20347506,  0.06659058, -0.19776642, -0.12739097,  0.5259511 ,
  

In [108]:
z = np.random.randn(64, 22)
print(z)
xhat = decoder(z)
print(xhat.mean())

[[-0.14212374  0.19793768 -1.73687961 ...  1.74948912  2.08180864
   0.04510323]
 [-1.60934631 -0.48827103 -0.0646957  ...  0.51671417 -0.10685113
  -0.1124632 ]
 [ 1.95227641 -1.02961352 -0.43261838 ...  0.19712715 -1.22698731
  -1.58822471]
 ...
 [ 0.89074244  0.35954555 -0.05539693 ...  1.75990446 -1.16922635
  -0.4335401 ]
 [-0.03511191  1.40166987 -0.21259589 ...  0.27623497 -0.45627774
  -1.15161641]
 [-1.44117815  2.51038397  0.95169061 ...  1.90799593  1.13793838
   1.36052546]]
tf.Tensor(
[[ 0.30800635  0.4006692   0.22512878 ... -0.16683182  0.13349782
  -0.36359137]
 [ 0.19616096  0.08947431  0.24414209 ...  0.02916822  0.01741795
   0.06638452]
 [ 0.1825361   0.39102787 -0.14198276 ... -0.16384827 -0.474496
  -0.3207624 ]
 ...
 [ 0.1258147   0.38306266  0.39623412 ...  0.12016383 -0.05660655
   0.03167184]
 [ 0.2670921  -0.09077801 -0.09012572 ... -0.14493103 -0.16807288
   0.0259023 ]
 [ 0.23974419 -0.22097726 -0.11254585 ... -0.1248567   0.57150745
  -0.02320774]], shape=